* http://cassandra.apache.org/doc/latest/getting_started/


* https://help.aiven.io/en/articles/1803299-getting-started-with-aiven-for-cassandra

In [ ]:
# https://github.com/datastax/python-driver

In [3]:
!pip install cassandra-driver

  Stored in directory: C:\Users\val-p1\AppData\Local\pip\Cache\wheels\08\43\84\50bd44f043b3c04c06b798cc5fc31d93586d38dfa3a48ec051
Successfully built geomet


In [4]:
from cassandra.cluster import Cluster

In [ ]:
cluster = Cluster()

In [5]:
# Copyright (c) 2018 Aiven, Helsinki, Finland. https://aiven.io/

import ssl

from cassandra.auth import PlainTextAuthProvider
from cassandra.cluster import Cluster
from cassandra.policies import DCAwareRoundRobinPolicy


def cassandra_example(args):
    auth_provider = PlainTextAuthProvider(args.username, args.password)
    ssl_options = {"ca_certs": args.ca_path, "cert_reqs": ssl.CERT_REQUIRED}
    with Cluster([args.host], port=args.port, ssl_options=ssl_options, auth_provider=auth_provider,
                 load_balancing_policy=DCAwareRoundRobinPolicy(local_dc='aiven')) as cluster:
        with cluster.connect() as session:
            # Create a keyspace
            session.execute("""
                CREATE KEYSPACE IF NOT EXISTS example_keyspace
                WITH REPLICATION = {'class': 'NetworkTopologyStrategy', 'aiven': 3}
            """)

            # Create a table
            session.execute("""
                CREATE TABLE IF NOT EXISTS example_keyspace.example_python (
                    id int PRIMARY KEY,
                    message text
                )
            """)

            # Insert some data
            for i in range(10):
                session.execute("""
                    INSERT INTO example_keyspace.example_python (id, message)
                    VALUES (%s, %s)
                """, (i, "Hello from Python!"))

            # Read it back
            for row in session.execute("SELECT id, message FROM example_keyspace.example_python"):
                print("Row: id = {}, message = {}".format(row.id, row.message))